In [ ]:
import pandas as pd
from pandas.io.common import EmptyDataError
from tqdm.autonotebook import trange, tqdm

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from datetime import datetime, timedelta, timezone
from pytz import timezone
import humanize

import os
import re

import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import time
import numpy as np
import math
import seaborn as sns

import requests
import io
import warnings

from collections import OrderedDict

In [ ]:
def import_df(filename, index = False):
    try:
        if index == False:
            df = pd.read_csv("Temp_Data/" + filename)
        else:
            df = pd.read_csv("Temp_Data/" + filename, index_col=0)
    except EmptyDataError:
        df = pd.DataFrame()
    return df

def set_date(date_str, old_tz, dt_format = "%d/%m/%Y %H:%M:%S"):
    if date_str == 'NaT':
        return pd.NaT
    else:
        datetime_set_naive = datetime.strptime(date_str, dt_format)
        datetime_set_old = timezone(old_tz).localize(datetime_set_naive)
        datetime_set_utc = datetime_set_old.astimezone(timezone('UTC'))
        return datetime_set_utc

In [ ]:
info = OrderedDict()
info['setup'] = import_df("info_setup.csv", index = True)
date_end = set_date(info['setup'].loc['date_end_utc','value'].replace('+00:00',''), "UTC", "%Y-%m-%d %H:%M:%S.%f")
diff = datetime.now(timezone('UTC')) - date_end
last_date = humanize.naturaldelta(diff)

In [ ]:
if diff > timedelta(hours = 1):
    import GetLatestData

In [ ]:
#Import data from temp_files
info = OrderedDict()
chart_dfs_mlt = []
plot_pars = []

for filename in tqdm(os.listdir("Temp_Data/"), desc = "Open files to import"):
        if filename.startswith('info_'):
            file_id = filename.replace('info_', '')
            file_id = file_id.replace('.csv', '')
            info[file_id] = import_df(filename, index = True)
        elif filename.startswith('chart_dfs_mlt' + '_'):
            chart_dfs_mlt.append(import_df(filename))
        elif filename.startswith('plot_pars' + '_'):
            plot_pars.append(import_df(filename, index = True))

charts = info['charts'].index.to_list()
date_end = set_date(info['setup'].loc['date_end_utc','value'].replace('+00:00',''), "UTC", "%Y-%m-%d %H:%M:%S.%f")

In [ ]:
chart_figs = []
for chart in charts:
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        chart_figs.append([])
        p = 0
        selected_plot_info = info['plots'].loc[info['plots'].index == chart][info['plots'].loc[info['plots'].index == chart]['plot'].isin(chart_dfs_mlt[chart].Plot.unique().tolist())]
        
        for plot in tqdm(selected_plot_info['plot'].to_list(), desc = "Creating plots for chart " + str(chart)):
            fig = go.Figure()
            fig_par = info['plots'].loc[info['plots'].index == chart].query('plot == "' + plot + '"')
            for i in range(0, len(chart_dfs_mlt[chart].query('Plot == "' + plot + '"').Parameter.unique())):
                par = chart_dfs_mlt[chart].query('Plot == "' + plot + '"').Parameter.unique()[i]
                plot_par = plot_pars[chart].query('parameter == "' + par + '"')
                x_data = chart_dfs_mlt[chart][chart_dfs_mlt[chart].Parameter == par].DateTime
                y_data = chart_dfs_mlt[chart][chart_dfs_mlt[chart].Parameter == par].Value
                y_error = chart_dfs_mlt[chart][chart_dfs_mlt[chart].Parameter == par].Error

                fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines',
                                        name=plot_par['parameter_lab'][0], 
                                        line=dict(color=plot_par['colour'][0], width=1),
                                        connectgaps=True, legendgroup=plot_par['parameter_lab'][0], showlegend=bool(plot_par['show_in_legend'][0])
                                        ))
            fig.update_yaxes(title_text=fig_par['ylab'][chart], range=[fig_par['ymin'][chart], fig_par['ymax'][chart]])
            fig.update_xaxes(showgrid=True, showticklabels=False)
            fig.update_layout(margin=dict(l=100, r=200, b=0, t=0, pad=10),
            xaxis=dict(fixedrange=True),
            template="simple_white")
            chart_figs[chart].append(fig)
    else:
        chart_figs.append("")

#del chart, fig, i, plot, par, plot_par, x_data, y_data, y_error

Creating plots for chart 2:  36%|███▌      | 5/14 [00:20<00:36,  4.08s/it]

In [ ]:
dcc_chart_figs = []
for chart in charts:
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        dcc_chart_figs.append([])
        p = 0
        for plot in range(0,len(chart_figs[chart])):
            dcc_chart_figs[chart].append(
                dcc.Graph(id='graph' + str(p),
                    figure=chart_figs[chart][plot],
                    style={'width': '98vw', 'height': '20vh'})
                )
            p = p + 1
    else:
        dcc_chart_figs.append("")

#del chart, p, plot

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
#external_stylesheets = [dbc.themes.LITERA]

app = dash.Dash(__name__, external_stylesheets = external_stylesheets)

tabs_init = []
for chart in charts:
    if info['charts'].loc[chart, 'chart_status'] == 'ON':
        tabs_init.append(dcc.Tab(label=info['charts']['chart_label'][chart], value="".join(["tab-", str(chart)])))

diff = datetime.now(timezone('UTC')) - date_end
last_date = humanize.naturaldelta(diff)
update_text = html.Div(html.P('Data last retrieved ' + last_date + ' ago'))

app.layout = html.Div(children=[
    html.Div([html.Img(src=app.get_asset_url('ToOL-PRO-BES_2.png'), style={'width':'90%', 'max-width': '100%'})], style={'textAlign': 'center'}),
    update_text,
    html.Div(children=[
        dcc.Tabs(id="tabs",
            value="tab-0",
            children=tabs_init),

        html.Div(children=[
            dcc.Loading(id='tabs-content')
        ]),
    ]),
])

@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    time.sleep(2)
    if tab == 'tab-0':
        return html.Div(id='loading-0', children=dcc_chart_figs[0])
    elif tab == 'tab-1':
        return html.Div(id='loading-1', children=dcc_chart_figs[1])
    elif tab == 'tab-2':
        return html.Div(id='loading-2', children=dcc_chart_figs[2])
    elif tab == 'tab-3':
        return html.Div(id='loading-3', children=dcc_chart_figs[3])

In [ ]:
if __name__ == '__main__':
    app.run_server()
    #debug=True, dev_tools_hot_reload_interval=5000)
                   #dev_tools_hot_reload_max_retry=30)

In [ ]:
tab1_content = dbc.Card(
    dbc.CardBody(
        [
            html.P("This is tab 1!", className="card-text"),
            dbc.Button("Click here", color="success"),
        ]
    ),
    className="mt-3",
)

tab2_content = dbc.Card(
    dbc.CardBody(
        [
            html.P("This is tab 2!", className="card-text"),
            dbc.Button("Don't click here", color="danger"),
        ]
    ),
    className="mt-3",
)


tabs = dbc.Tabs(
    [
        dbc.Tab(tab1_content, label="Tab 1"),
        dbc.Tab(tab2_content, label="Tab 2"),
        dbc.Tab(
            "This tab's content is never seen", label="Tab 3", disabled=True
        ),
    ]
)

layout = html.Div([tabs])

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LITERA]

app.layout = layout